In [1]:
%%time
import pandas as pd
df = pd.read_excel('D:/Work/Course Work/Semester 4/DS Competition/Data/New folder/newtable.xlsx',sheet_name='newtable')

df.head()
df.dtypes

Wall time: 11min 20s


trip_id                  int64
bike_id                 object
start_station            int64
trip_route_category     object
start_time              object
end_time                object
start_lat              float64
start_lon              float64
plan_duration            int64
passholder_type         object
trip_duration            int64
end_station              int64
end_lat                float64
end_lon                float64
dtype: object

In [2]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
from datetime import datetime,date
from sklearn.model_selection import train_test_split
from matplotlib import pyplot 
from sklearn.metrics import mean_squared_error
from math import sqrt


In [78]:
ial=[3021,	3053,	3055,	3059,	
     3060,	3061,	3079,	3080,	
     4108,	4138,	4142,	4143,	
     4144,	4146,	4147,	4148,	
     4149,	4150,	4151,	4152,	
     4153,	4154,	4155,	4156,	
     4157,	4158,	4159,	4160,	
     4162,	4163,	4165,	4166,	
     4167,	4169,	4170,	4174,	
     4176,	4177,	4180,	4181,	
     4183,	4194,	4244,  4276] 
df=df[~df.start_station.isin(ial)]

In [79]:
df_d=pd.pivot_table(df[['start_station','start_time']],aggfunc='count',index=df['start_time'].dt.date,columns=['start_station'],fill_value=0)  #date

df_d.index = pd.to_datetime(df_d.index)
df_d.head()

start_time                                               ...  \
start_station       3005 3006 3007 3008 3009 3010 3011 3013 3014 3016  ...   
start_time                                                             ...   
2016-07-07             9    8    1    0    0    0    0    0   14    9  ...   
2016-07-08            20   19    5    2    0    0    0    0   22   10  ...   
2016-07-09            21   11    1    4    0    0    0    0    8    1  ...   
2016-07-10            10   17    5    6    0    2    0    0    5    0  ...   
2016-07-11             9   12    3    5    0    1    0    0   19   13  ...   

                                                                 
start_station 4245 4246 4248 4249 4250 4254 4266 4267 4273 4275  
start_time                                                       
2016-07-07       0    0    0    0    0    0    0    0    0    0  
2016-07-08       0    0    0    0    0    0    0    0    0    0  
2016-07-09       0    0    0    0    0    0    0    0    0    0  
2016-07-10       0    0    0    0    0    0    0    0    0    0  
2016-07-11       0    0    0    0    0    0    0    0    0    0  

[5 rows x 96 columns]

In [80]:
df_o=pd.DataFrame()
mse_trn=[]
mse_tst=[]


In [81]:
for i,v in enumerate(df_d.columns):
    #i=0
    #v=('start_time', 3005)
    
    def prophet_inputize(df,column_num = 0):
        df_1 = pd.DataFrame()
        df_1['ds'] = df.index
        df_1['y'] = list(df.iloc[:,i])
        return(df_1)
    
    def do_something(df_d,sample ='D',test =-184):
        
        
        #df_d = df_d.resample(sample).sum()
        
        # imported from the cycling event url:https://www.ciclavia.org/events_history
        rem=['2016-08-14','2018-10-16','2017-03-26','2017-05-11','2017-08-13','2017-10-08',\
           '2017-12-10','2018-04-22','2018-06-24','2018-09-30','2018-12-02']

        df_d['temp']=df_d.index   # creating a column of indexes

        for l in rem:
            df_d=df_d[df_d.temp!=l]            # removing the rows with rem values as these are the outliers
    

        df_d.drop('temp',axis=1,inplace=True)  # removing the index column
        
        
        df_d_trn = pd.DataFrame()
        df_d_tst = pd.DataFrame()
        
        df_d_trn = df_d.loc[df_d.index[:test]]#int(train_prop*len(df_d.index))]]
                
        df_d_tst = df_d.loc[df_d.index[test:]]#int(train_prop*len(df_d.index)):]]
    
        df_d_trn = prophet_inputize(df_d_trn,i)
        df_d_tst = prophet_inputize(df_d_tst,i)
    
#        df_d_trn.plot(x='ds',y='y')
#        df_d_tst.plot(x='ds',y='y')
        
        return(df_d_trn,df_d_tst)

    
    sample_freq = 'D'
    trn, tst = do_something(df_d,sample_freq,-184)    
    m = Prophet(yearly_seasonality=True,weekly_seasonality= False,daily_seasonality=False)
    m.fit(trn)
    future = m.make_future_dataframe(periods=len(tst)+90+3,freq=sample_freq)    # till 31st march 2019 q1 2019
    forecast = m.predict(future)
    df_o[v]=forecast['yhat']
#    df_o['ds']=forecast['ds']
#    fig1 = m.plot(forecast)
#    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())
#    print(tst.tail())
    y_actual_tst=tst['y']
    y_predicted_tst=df_o[v][df_o.index[-184:]]
    mse_tst.append(sqrt(mean_squared_error(y_actual_tst, y_predicted_tst)))
    
    y_actual_trn=trn['y']
    y_predicted_trn=df_o[v][df_o.index[:-184-93]]                              ## removing q1 2019 dates
    mse_trn.append(sqrt(mean_squared_error(y_actual_trn, y_predicted_trn)))

df_o['ds']=forecast['ds']

In [82]:
### now run this


# imported from the cycling event url:https://www.ciclavia.org/events_history
rem=['2016-08-14','2018-10-16','2017-03-26','2017-05-11','2017-08-13','2017-10-08',\
           '2017-12-10','2018-04-22','2018-06-24','2018-09-30','2018-12-02']

df_d['ds']=df_d.index   # creating a column of indexes

for l in rem:
    df_d=df_d[df_d.ds!=l]            # removing the rows with rem values as these are the outliers
    
df_d=df_d.reset_index(drop=True)     # reset index to zero

mse_trn=pd.DataFrame(mse_trn)
mse_tst=pd.DataFrame(mse_tst)


In [83]:
with pd.ExcelWriter('C:\\Users\\mayan\\Downloads\\New folder\\outputY.xlsx') as writer:  # doctest: +SKIP
    df_d.to_excel(writer,sheet_name='actual(Y)')
    df_o.to_excel(writer,sheet_name='predicted(Y)')
    mse_trn.to_excel(writer,sheet_name='mse_trn(Y)')
    mse_tst.to_excel(writer,sheet_name='mse_tst(Y)')

In [87]:
df_d.head()

start_time                                               ...  \
start_station       3005 3006 3007 3008 3009 3010 3011 3013 3014 3016  ...   
0                      9    8    1    0    0    0    0    0   14    9  ...   
1                     20   19    5    2    0    0    0    0   22   10  ...   
2                     21   11    1    4    0    0    0    0    8    1  ...   
3                     10   17    5    6    0    2    0    0    5    0  ...   
4                      9   12    3    5    0    1    0    0   19   13  ...   

                                                            temp         ds  
start_station 4248 4249 4250 4254 4266 4267 4273 4275                        
0                0    0    0    0    0    0    0    0 2016-07-07 2016-07-07  
1                0    0    0    0    0    0    0    0 2016-07-08 2016-07-08  
2                0    0    0    0    0    0    0    0 2016-07-09 2016-07-09  
3                0    0    0    0    0    0    0    0 2016-07-10 2016-07-10  
4                0    0    0    0    0    0    0    0 2016-07-11 2016-07-11  

[5 rows x 98 columns]

In [115]:
df_d['y']=df_d.drop([('temp','')],axis=1).sum(axis=1)

In [116]:
df_d.head()

start_time                                               ...  \
start_station       3005 3006 3007 3008 3009 3010 3011 3013 3014 3016  ...   
0                      9    8    1    0    0    0    0    0   14    9  ...   
1                     20   19    5    2    0    0    0    0   22   10  ...   
2                     21   11    1    4    0    0    0    0    8    1  ...   
3                     10   17    5    6    0    2    0    0    5    0  ...   
4                      9   12    3    5    0    1    0    0   19   13  ...   

                                                       temp         ds    y  
start_station 4249 4250 4254 4266 4267 4273 4275                             
0                0    0    0    0    0    0    0 2016-07-07 2016-07-07  173  
1                0    0    0    0    0    0    0 2016-07-08 2016-07-08  323  
2                0    0    0    0    0    0    0 2016-07-09 2016-07-09  373  
3                0    0    0    0    0    0    0 2016-07-10 2016-07-10  310  
4                0    0    0    0    0    0    0 2016-07-11 2016-07-11  329  

[5 rows x 99 columns]

In [130]:
t=pd.DataFrame()
t['ds']=df_d[('ds','')]
t['y']=df_d['y']
t.head()

,ds,y
0,2016-07-07,173
1,2016-07-08,323
2,2016-07-09,373
3,2016-07-10,310
4,2016-07-11,329


In [136]:
t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 897 entries, 0 to 896
Data columns (total 2 columns):
ds    897 non-null datetime64[ns]
y     897 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 14.1 KB


In [134]:
t.iloc[0:25,:]

,ds,y
0,2016-07-07,173
1,2016-07-08,323
2,2016-07-09,373
3,2016-07-10,310
4,2016-07-11,329
5,2016-07-12,496
6,2016-07-13,490
7,2016-07-14,535
8,2016-07-15,449
9,2016-07-16,374


In [137]:
#dft1=pd.DataFrame(list(df_d['index']),columns=['ds'])
#dft1.rename(columns={'0':'ds'}, inplace=True)
#dft1['y']=list(df_d.loc[:,'y'])
#dft1.head()

dft2=t.iloc[0:25,:]
dft2.tail()
dftv=t.iloc[-25:,:]
dftv.head()
#dft2.plot(x='ds',y='y')
#dftv.plot(x='ds',y='y')


m=Prophet()
m.fit(dft2)

future = m.make_future_dataframe(periods=25)
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 19.


,ds,yhat,yhat_lower,yhat_upper
45,2016-08-21,604.424008,524.236307,687.026330
46,2016-08-22,670.184296,591.067802,754.901835
47,2016-08-23,725.854376,641.128064,806.945030
48,2016-08-24,742.849718,656.769426,828.425456
49,2016-08-25,718.239642,640.211901,798.424120


In [138]:
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2016-07-07,334.409741,266.021962,430.581381,334.409741,334.409741,14.451948,14.451948,14.451948,14.451948,14.451948,14.451948,0.0,0.0,0.0,348.861689
1,2016-07-08,341.948067,284.600099,456.291473,341.948067,341.948067,27.909842,27.909842,27.909842,27.909842,27.909842,27.909842,0.0,0.0,0.0,369.857908
2,2016-07-09,349.486392,224.073216,386.221562,349.486392,349.486392,-45.906666,-45.906666,-45.906666,-45.906666,-45.906666,-45.906666,0.0,0.0,0.0,303.579727
3,2016-07-10,357.024718,199.640745,370.169550,357.024718,357.024718,-69.210384,-69.210384,-69.210384,-69.210384,-69.210384,-69.210384,0.0,0.0,0.0,287.814334
4,2016-07-11,364.563043,275.656335,435.155334,364.563043,364.563043,-10.988422,-10.988422,-10.988422,-10.988422,-10.988422,-10.988422,0.0,0.0,0.0,353.574622
5,2016-07-12,372.101369,330.120363,492.957721,372.101369,372.101369,37.143333,37.143333,37.143333,37.143333,37.143333,37.143333,0.0,0.0,0.0,409.244702
6,2016-07-13,379.639695,347.423205,503.383930,379.639695,379.639695,46.600349,46.600349,46.600349,46.600349,46.600349,46.600349,0.0,0.0,0.0,426.240044
7,2016-07-14,387.178020,325.891711,485.410078,387.178020,387.178020,14.451948,14.451948,14.451948,14.451948,14.451948,14.451948,0.0,0.0,0.0,401.629968
8,2016-07-15,394.716346,348.421435,505.659602,394.716346,394.716346,27.909842,27.909842,27.909842,27.909842,27.909842,27.909842,0.0,0.0,0.0,422.626187
9,2016-07-16,402.254671,278.321718,440.167074,402.254671,402.254671,-45.906666,-45.906666,-45.906666,-45.906666,-45.906666,-45.906666,0.0,0.0,0.0,356.348006


In [ ]:

###############################################################################

#month analysis
df_m['index']=df_m.index
dft1 = pd.DataFrame(list(df_m['index']))#,columns=['ds'])
#dft1.rename(columns={'0':'year','1':'month'}, inplace=True)

dft1['y']=list(df_m.iloc[:,0])
dft1.head()

df_m.columns[0]
df_m.plot(x='index',y=df_m.columns[0])

dft2=dft1.iloc[0:-25,:]
dft2.tail()
dftv=dft1.iloc[-25:,:]
dftv.head()
dft2.plot(x='ds',y='y')
dftv.plot(x='ds',y='y')


m=Prophet()
m.fit(dft2)

future = m.make_future_dataframe(periods=25)
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

fig1 = m.plot(forecast)
